In [3]:
!pip install folium

In [4]:
!pip install geopy

In [5]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from math import radians, cos, sin, sqrt, atan2
from IPython.display import display
from folium import Popup

In [6]:
# 1. 하버사인 공식을 이용한 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반지름 (단위: km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [8]:
# 2. 주소를 위경도로 변환하는 함수
def get_lat_lon(address):
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError("주소를 찾을 수 없습니다. 정확한 주소를 입력하세요.")

In [9]:
# 3. 사용자 위치 입력
while True:
    try:
        print("나의 위치를 입력해주세요 (예: 서울특별시 중구 세종대로 110):")
        my_address = input("주소: ")  # 사용자로부터 주소 입력받기
        my_lat, my_lon = get_lat_lon(my_address)  # 위경도 변환
        print(f"입력된 주소의 위경도: {my_lat}, {my_lon}")
        break  # 에러가 없으면 루프 종료
    except ValueError as e:
        print(f"에러: {e}. 다시 시도해주세요.")  # 에러 메시지 출력 후 재시도

나의 위치를 입력해주세요 (예: 서울특별시 중구 세종대로 110):
주소: 서울특별시 중구 세종대로 110
입력된 주소의 위경도: 37.56635025, 126.97830900276097


In [10]:
# 4. 데이터 읽기 및 하버사인 거리 계산
file_path = '유통센터_공판장_도매시장_정리.csv'
data = pd.read_csv(file_path, encoding='euc-kr')

# 위경도 컬럼 이름 통일
data.rename(columns={'위도': '위도', '경도': '경도', '종류': '종류', '명칭': '명칭'}, inplace=True)

# 거리 계산
data['거리'] = data.apply(
    lambda row: haversine(my_lat, my_lon, row['위도'], row['경도']), axis=1
)

In [12]:
# 5. 새로운 필터링 로직
max_distances = {'유통센터': 50, '공판장': 35, '도매시장': 20}

def filter_or_nearest(group, type_name):
    filtered = group[group['거리'] <= max_distances[type_name]]
    if not filtered.empty:
        return filtered.nsmallest(3, '거리')
    # 범위 내 유통센터가 없을 경우 가장 가까운 한 곳 추천
    return group.nsmallest(1, '거리')

filtered_top_3_by_type = data.groupby('종류', group_keys=False).apply(lambda group: filter_or_nearest(group, group.name)).reset_index(drop=True)

filtered_top_3_by_type

,종류,명칭,주소,위도,경도,시도구분,시도시군구,법정동코드,전화번호,거리
0,공판장,동두천농협,강원특별자치도 동두천시 중앙로 207,37.899876,127.052241,강원특별자치도,동두천시,4125000000,NaN,37.651982
1,도매시장,서울강서농수산물도매시장,서울특별시 강서구 발산로40,37.553629,126.821398,서울특별시,강서구,1150000000,(02)2640-6000,13.903203
2,도매시장,서울가락농수산물도매시장,서울특별시 송파구 양재대로 932,37.493482,127.111504,서울특별시,송파구,1171000000,(02)3435-1000,14.268970
3,도매시장,구리농수산물도매시장,경기도 구리시 동구릉로 136번길 90,37.611932,127.144917,경기도,구리시,4131000000,(031)560-5100,15.530393
4,유통센터,화성시 농산물 산지유통센터(APC),화성시 팔탄면 푸른들판로 526,37.155560,126.900026,경기도,화성시,4159000000,031-5180-7810,46.198795
5,유통센터,군자농협 APC,경기도 안산시 단원구 대부중앙로 141(대부동동),37.241490,126.589279,경기도,안산시,4127000000,032-886-6100,49.856513


In [13]:
# 지도 생성
m = folium.Map(location=[my_lat, my_lon], zoom_start=12)

#나의 위치 표시
my_location_popup = folium.Popup(
    f"""
    <b>나의 위치</b><br>
    <b>주소:</b> {my_address}
    """, 
    max_width=300
)


# 나의 위치 표시
folium.Marker(
    location=[my_lat, my_lon],
    popup=my_location_popup,
    tooltip="나의 위치",
    icon=folium.Icon(color="red")
).add_to(m)

In [15]:
# 상위 3개씩 마커 추가
for _, row in filtered_top_3_by_type.iterrows():
    # 종류별 색상 설정
    if row['종류'] == "공판장":
        marker_color = "green"
    elif row['종류'] == "유통센터":
        marker_color = "blue"
    else:
        marker_color = "gray"

    #팝업 설정
    popup_content = f"""
    <b>이름:</b> {row['명칭']}<br>
    <b>종류:</b> {row['종류']}<br>
    <b>거리:</b> {row['거리']:.2f} km<br>
    <b>주소:</b> {row['주소']}
    """
    # 전화번호가 있을 경우 추가
    if not pd.isna(row.get('전화번호')) and row['전화번호']:
        popup_content += f"<br><b>전화번호:</b> {row['전화번호']}"
    
    popup = folium.Popup(popup_content, max_width=300)

    # 마커 추가
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=popup,
        tooltip=f"거리: {row['거리']:.2f} km",
        icon=folium.Icon(color=marker_color)
    ).add_to(m)

In [16]:
from IPython.display import display, HTML

# Folium 지도 객체 렌더링
display(m)

# 데이터 표 생성 및 렌더링
table_html = filtered_top_3_by_type.to_html(index=False, escape=False)

# 표를 Jupyter Notebook에서 바로 표시
display(HTML(f"""
    <h2>추천된 장소 목록</h2>
    {table_html}
"""))


종류,명칭,주소,위도,경도,시도구분,시도시군구,법정동코드,전화번호,거리
공판장,동두천농협,강원특별자치도 동두천시 중앙로 207,37.899876,127.052241,강원특별자치도,동두천시,4125000000,NaN,37.651982
도매시장,서울강서농수산물도매시장,서울특별시 강서구 발산로40,37.553629,126.821398,서울특별시,강서구,1150000000,(02)2640-6000,13.903203
도매시장,서울가락농수산물도매시장,서울특별시 송파구 양재대로 932,37.493482,127.111504,서울특별시,송파구,1171000000,(02)3435-1000,14.268970
도매시장,구리농수산물도매시장,경기도 구리시 동구릉로 136번길 90,37.611932,127.144917,경기도,구리시,4131000000,(031)560-5100,15.530393
유통센터,화성시 농산물 산지유통센터(APC),화성시 팔탄면 푸른들판로 526,37.155560,126.900026,경기도,화성시,4159000000,031-5180-7810,46.198795
유통센터,군자농협 APC,경기도 안산시 단원구 대부중앙로 141(대부동동),37.241490,126.589279,경기도,안산시,4127000000,032-886-6100,49.856513
